# Natural Language Processing - Assignment 2
# Sentiment analysis for movie reviews

This notebook was created for you to answer question 2, 3 and 4 from assignment 2. Please read the steps and the provided code carefully and make sure you understand them. 

The (red) comments at the beginning of each function explain what they should do, which parameters you should give as input and which variables should be returned by the function. After the (green) comments "### student code here###' you should write your own code.

**Please modify the next cell specifying your group number**

 *This is the Notebook of* ***Group 30*** 




### Prerequisite - Libraries
Make sure you have the needed libraries installed on your computer: scikit-learn, Pandas, NLTK...

### Prerequisite - Load Data

In the first step, we are going to load the data in a Pandas DataFrame. Pandas DataFrames are a useful way of storing data. DataFrames are tables in which data can be accessed as columns, as rows or as individual cells. You can find more info on DataFrames here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

Read the code below and make sure you understand what is happening. Run the code to load your data.

In [15]:
import os
import re
import pandas as pd
import numpy as np
import glob
### student code here: import the needed modules from sci-kit learn ###
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer as CountVectorizerOff
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score
from collections import defaultdict, Counter
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to C:\Users\Marko
[nltk_data]     Haralović\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Marko
[nltk_data]     Haralović\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
def get_path(filename):
    """
    Makes a list of all the paths that fit the search requirement
    
    :param filename: A regular expression that defines the search requirement for the filenames
    :return  Returns a list of all the pathnames
    """
    # place the movies folder in the same directory as this notebook
    current_directory = os.getcwd()
    # if you are using Google Colab, you will have to change the above line
    # to load the dataset from your Google Drive

    # glob.glob() is a pattern-matching path finder, it searches for the reviews in the movies folder based on a Regular Expression
    paths = glob.glob(current_directory + '/movies/' + filename)
    
    if len(paths) == 0:
        print('Your file list is empty. The code looks for the folder '+current_directory+'/movies, but could not find it.')
    else: 
        print("Found ", len(paths), "files")
    return paths

In [17]:
def load_data(pathset):
    """
    Loads the data into a dataframe
    
    :param pathset:  A list of paths
    :return  A dataframe with three columns: Path, Review (Text) and Label
    """
    # Files are named by sentiment (P for positive, N for negative)
    pattern = re.compile('P-(train|test)[0-9]*.txt')
    reviews = []
    labels = []
    df = pd.DataFrame(columns = ['Path', 'Review', 'Label'])
    for path in pathset:
        if re.search(pattern, path):
            text = open(path, "r").read()
            reviews.append(text)
            labels.append('Pos')
        else:
            text = open(path, "r").read()
            reviews.append(text)
            labels.append('Neg')
    df['Path'] = pathset
    df['Review'] = reviews
    df['Label'] = labels
    return df

In [18]:
#Load the files in the Dataframe. This will take a while...
paths = get_path('train/[NP]-train[0-9]*.txt')
data = load_data(paths)
data.head()

Found  1200 files


,Path,Review,Label
0,c:\FER\9TH SEMESTER\NLP\homeworks\hw2/movies/t...,Once again Mr. Costner has dragged out a movie...,Neg
1,c:\FER\9TH SEMESTER\NLP\homeworks\hw2/movies/t...,This is a pale imitation of 'Officer and a Gen...,Neg
2,c:\FER\9TH SEMESTER\NLP\homeworks\hw2/movies/t...,"It seems ever since 1982, about every two or t...",Neg
3,c:\FER\9TH SEMESTER\NLP\homeworks\hw2/movies/t...,"Wow, another Kevin Costner hero movie. Postman...",Neg
4,c:\FER\9TH SEMESTER\NLP\homeworks\hw2/movies/t...,"Alas, another Costner movie that was an hour t...",Neg


### Part 2 - Tokenization

In this step, you should write a tokenizer and compare it with an off-the-shelf one.

#### Question 2.1 Making your own tokenizer

In [19]:
def my_tokenizer(text):
    """
    The implementation of your own tokenizer
    
    :param text:  A string with a sentence (or paragraph, or document...)
    :return  A list of tokens
    """    
    ### student code here ###
    
    # part 1: regex to find the words
    regex = r'[a-zA-Z]+(?:\.[a-zA-Z]+)*|\d+|[.!?,:;()"\'](?=\s)'
    words = re.findall(regex, text.lower())
    
    # part 2: lemmatization
    lematized_words = [WordNetLemmatizer().lemmatize(word) if word.isalpha() else word for word  in words]

    tokenized_text = lematized_words
    return tokenized_text

sample_string0 = "If you have the chance, watch it. Although, a warning, you'll cry your eyes out."
sample_string1 = "I don't know what to do with this course I've taken, it has all these assignments..."
sample_string2 = "I tried to like this movie, I really did, but it was just so bad." 
sample_string3 = "No matter what I do, my girlfriend is always right. I decided to acccept it"
print(my_tokenizer(sample_string0))
print(my_tokenizer(sample_string1))
print(my_tokenizer(sample_string2))
print(my_tokenizer(sample_string3))

['if', 'you', 'have', 'the', 'chance', ',', 'watch', 'it', '.', 'although', ',', 'a', 'warning', ',', 'you', 'll', 'cry', 'your', 'eye', 'out']
['i', 'don', 't', 'know', 'what', 'to', 'do', 'with', 'this', 'course', 'i', 've', 'taken', ',', 'it', 'ha', 'all', 'these', 'assignment']
['i', 'tried', 'to', 'like', 'this', 'movie', ',', 'i', 'really', 'did', ',', 'but', 'it', 'wa', 'just', 'so', 'bad']
['no', 'matter', 'what', 'i', 'do', ',', 'my', 'girlfriend', 'is', 'always', 'right', '.', 'i', 'decided', 'to', 'acccept', 'it']


#### Question 2.2 Using an off-the-shelf tokenizer

In [20]:
#Now we are gonna compare the tokenizer you just wrote with the one from NLTK
#if you installed NLTK but never downloaded the 'punkt' tokenizer, uncomment the following lines:

def nltk_tokenizer(text):
    """
    This function should apply the word_tokenize (punkt) tokenizer of nltk to the input text
    
    :param text:  A string with a sentence (or paragraph, or document...)
    :return  A list of tokens
    """     
    ### student code here ###    
    tokenized_text = word_tokenize(text.lower())
    
    return tokenized_text

test_sentences = ["I like this assignment because:\n-\tit is fun;\n-\tit helps me practice my Python skills.",
        "I won a prize, but I won't be able to attend the ceremony.",
        "“The strange case of Dr. Jekyll and Mr. Hyde” is a famous book... but I haven't read it.",
        "I work for the C.I.A.. And you?",
        "OMG #Twitter is sooooo coooool <3 :-) <-- lol...why do i write like this idk right? :) 🤷😂 🤖"]

for test_string in test_sentences:
    print(my_tokenizer(test_string))
    print(nltk_tokenizer(test_string))
    print("\n")
    

['i', 'like', 'this', 'assignment', 'because', ':', 'it', 'is', 'fun', ';', 'it', 'help', 'me', 'practice', 'my', 'python', 'skill']
['i', 'like', 'this', 'assignment', 'because', ':', '-', 'it', 'is', 'fun', ';', '-', 'it', 'helps', 'me', 'practice', 'my', 'python', 'skills', '.']


['i', 'won', 'a', 'prize', ',', 'but', 'i', 'won', 't', 'be', 'able', 'to', 'attend', 'the', 'ceremony']
['i', 'won', 'a', 'prize', ',', 'but', 'i', 'wo', "n't", 'be', 'able', 'to', 'attend', 'the', 'ceremony', '.']


['the', 'strange', 'case', 'of', 'dr', '.', 'jekyll', 'and', 'mr', '.', 'hyde', 'is', 'a', 'famous', 'book', '.', 'but', 'i', 'haven', 't', 'read', 'it']
['“', 'the', 'strange', 'case', 'of', 'dr.', 'jekyll', 'and', 'mr.', 'hyde', '”', 'is', 'a', 'famous', 'book', '...', 'but', 'i', 'have', "n't", 'read', 'it', '.']


['i', 'work', 'for', 'the', 'c.i.a', '.', 'and', 'you']
['i', 'work', 'for', 'the', 'c.i.a', '..', 'and', 'you', '?']


['omg', 'twitter', 'is', 'sooooo', 'coooool', '3', ')', '

### Part 3 - Text classification with a unigram language model

#### Training phase
You now need to create the model and train it on the documents in the dataframe. Look at the scikit learn documentation to learn how to use the CountVectorizer and MultimodalNaiveBayes modules.

In [21]:
class CountVectorizer:
   def __init__(self, regex = r"\b\w+\b"):
      self.vocabulary = defaultdict(int)
      self.regex = re.compile(regex)
      self.feature_names = []
      
   def tokenize(self, text):
      text = self.lowercase(text)
      return re.findall(self.regex, text)
   
   def lowercase(self, text):
      return text.lower()
   
   def empty_vocab(self):
      self.vocabulary = {}
      
   def fit(self, documents):
      self.empty_vocab()
      for doc in documents:
         for word in set(self.tokenize(doc)):
            if word not in self.vocabulary:
               self.vocabulary[word] = len(self.vocabulary)
               
      self.feature_names = [None]  * len(self.vocabulary)
      for word, index in self.vocabulary.items():
         self.feature_names[index] =  word
      return self
   
   def transform(self, documents):
      X = np.zeros((len(documents), len(self.vocabulary)), dtype=np.int32)
      for i, doc in enumerate(documents):
         counts = Counter(self.tokenize(doc))
         for word, count in counts.items():
            index = self.vocabulary.get(word)
            if index is not None:
               X[i, index] = count
               
      return X
  
   def fit_transform(self, documents):
      return self.fit(documents).transform(documents)
   
   def  get_feature_names(self):
      return np.array(self.feature_names)
   

In [22]:
class MultiModalNaiveBayes:
    def __init__(self, alpha=1.0, class_prior=None):
        self.alpha = float(alpha)
        self.class_prior = class_prior
        self.classes_ = None
        self.class_priors_ = class_prior  
        self.likelihoods_ = None 

    def fit(self, X, y):
        y = np.asarray(y)
        self.classes_ = np.unique(y)

        if self.class_prior is not None:
            priors = self.class_prior / self.class_prior.sum()
        else:
            counts = np.array([(y == c).sum() for c in self.classes_], dtype=float)
            priors = counts / counts.sum()
        self.class_priors_ = priors

        self.likelihoods_ = self._calculate_likelihoods(X, y)
        return self

    def _calculate_likelihoods(self, X, y):
        L = np.zeros((self.classes_.shape[0],  X.shape[1]), dtype=float)

        for i, c in enumerate(self.classes_):
            X_c = X[y == c]                       
            word_counts = np.asarray(X_c.sum(axis=0))
            L[i, :] =(word_counts + self.alpha) / (word_counts.sum() + self.alpha * X.shape[1])
        return L

    def predict_proba(self, X):
        log_prior = np.log(self.class_priors_)[None, :]              
        log_lik_T = np.log(self.likelihoods_).T                      
        log_scores = log_prior + X @ log_lik_T                     

        log_scores -= log_scores.max(axis=1, keepdims=True)
        probs = np.exp(log_scores)
        probs /= probs.sum(axis=1, keepdims=True)
        return probs

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


In [23]:
# First, read all the test data from the files.  
# Then classify it using the classifier you trained before
# Finally, calculate the performance

## Our implementation of vectorizer and bayesian network

print("OUR IMPLEMENTATION RESULTS")

test_paths = get_path('test/[NP]-test[0-9]*.txt')
test_data = load_data(test_paths)
X_train = data['Review']
y_train = data['Label']
X_test = test_data['Review']
y_test = test_data['Label']

# w laplace
model = MultiModalNaiveBayes(alpha=1)
vectorizer = CountVectorizer()

X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)

accuracy = sum(predictions == y_test) / len(y_test)*100
print(f"Accuracy with laplace smoothing and no stopwords removal: {accuracy:.2f}%")

# without laplace
model = MultiModalNaiveBayes(alpha=0.0)
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)

accuracy = sum(predictions == y_test) / len(y_test)*100
print(f"Accuracy without laplace smoothing and no stopwords removal: {accuracy:.2f}%")

#Model with stop words removed:

model = MultiModalNaiveBayes(alpha=1.0)

vectorizer = CountVectorizerOff(stop_words="english")
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)

accuracy = sum(predictions == y_test) / len(y_test)*100
print(f"Accuracy with laplace smoothing and stop words removed : {accuracy:.2f}%")


OUR IMPLEMENTATION RESULTS
Found  100 files
Accuracy with laplace smoothing and no stopwords removal: 82.00%


C:\Users\Marko Haralović\AppData\Local\Temp\ipykernel_30704\2972429044.py:34: RuntimeWarning: divide by zero encountered in log
  log_lik_T = np.log(self.likelihoods_).T
C:\Users\Marko Haralović\AppData\Local\Temp\ipykernel_30704\2972429044.py:35: RuntimeWarning: invalid value encountered in matmul
  log_scores = log_prior + X @ log_lik_T


Accuracy without laplace smoothing and no stopwords removal: 50.00%
Accuracy with laplace smoothing and stop words removed : 85.00%


In [29]:
## Comparison witht he official implementations

print("OFFICIAL IMPLEMENTATION RESULTS")


test_paths = get_path('test/[NP]-test[0-9]*.txt')
test_data = load_data(test_paths)
X_train = data['Review']
y_train = data['Label']
X_test = test_data['Review']
y_test = test_data['Label']

# with laplace smoothing and no stopwords removal
model = MultinomialNB(alpha=1.0)
vectorizer = CountVectorizerOff()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy_basic = sum(predictions == y_test) / len(y_test)*100

print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"Accuracy with laplace smoothing and no stopwords removal : {accuracy_basic:.2f}%")

# without laplace smoothing and no stopwords removal
model = MultinomialNB(alpha=0.0)
vectorizer = CountVectorizerOff()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy = sum(predictions == y_test) / len(y_test)*100

print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"Accuracy  without laplace smoothing and no stopwords removal: {accuracy:.2f}%")

# with laplace smoothing and with stopwords removal
model = MultinomialNB(alpha=1.0)
vectorizer = CountVectorizerOff(stop_words="english")
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy_basic = sum(predictions == y_test) / len(y_test)*100

print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"Accuracy with laplace smoothing and with stopwords removal: {accuracy_basic:.2f}%")

# with laplace smoothing and without stopwords removal and without lowercase
model = MultinomialNB(alpha=1.0)
vectorizer = CountVectorizerOff(lowercase=False)
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy = sum(predictions == y_test) / len(y_test)*100

print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"Accuracy with laplace smoothing and no stopwords removal and without lowercasing: {accuracy:.2f}%")

OFFICIAL IMPLEMENTATION RESULTS
Found  100 files
Vocabulary size: 17989
Accuracy with laplace smoothing and no stopwords removal : 84.00%


c:\Python311\Lib\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


Vocabulary size: 17989
Accuracy  without laplace smoothing and no stopwords removal: 50.00%
Vocabulary size: 17690
Accuracy with laplace smoothing and with stopwords removal: 85.00%
Vocabulary size: 20713
Accuracy with laplace smoothing and no stopwords removal and without lowercasing: 81.00%


Q: The performance after disabling the default lowercase normalization (and
without stop word removal). Is there a difference, and if so, why do you
think there is one?
A: The performance decreased slightly after disabling the default lowercase normalization. This is likely because the model can no longer treat words with different cases as the same word, leading to a larger vocabulary and sparser data, whcih can negatively impact the model's ability to learn (and generalize). It works a bit better (1pp) with stopwords removal, signaling that signal gotten from stopwords is marginally useful for this task.

### Part 4 - Text classification with a bigram language model

Now we will classify the same dataset again, but this time with a bigram language model. 

#### Training phase
Build a Naïve Bayes classifier that uses bigrams instead of single words.


In [25]:
### Student code here ###

class NgramModalNaiveBayes:
   def __init__(self, alpha=1.0, class_prior=None):
      self.alpha = float(alpha)
      self.class_prior = class_prior
      self.classes_ = None
      self.likelihoods_ = None 
        
   def fit(self, X, y):
      y = np.asarray(y)
      self.classes_ = np.unique(y)

      if self.class_prior is not None:
         priors = self.class_prior / self.class_prior.sum()
      else:
         counts = np.array([(y == c).sum() for c in self.classes_], dtype=float)
         priors = counts / counts.sum()
      self.class_priors_ = priors

      self.likelihoods_ = self._calculate_likelihoods(X, y)
      return self
         
   def _calculate_likelihoods(self, X, y):
      L = np.zeros((self.classes_.shape[0],  X.shape[1]), dtype=float)
      for i, c in enumerate(self.classes_):
         X_c = X[y == c]                       
         word_counts = np.asarray(X_c.sum(axis=0))
         L[i, :] = (word_counts + self.alpha) / (word_counts.sum() + self.alpha * X.shape[1])
      return L

   def predict_proba(self, X):
      log_prior = np.log(self.class_priors_)[None, :]              
      log_lik_T = np.log(self.likelihoods_).T                      
      log_scores = log_prior + X @ log_lik_T                     

      log_scores -= log_scores.max(axis=1, keepdims=True)
      probs = np.exp(log_scores)
      probs /= probs.sum(axis=1, keepdims=True)
      return probs

   def predict(self, X):
      return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


In [ ]:
### Student code here ###
vectorizer_bigram = CountVectorizerOff(ngram_range=(2, 2))

X_train_bigram = vectorizer_bigram.fit_transform(X_train)
print(f"Bigram vocabulary size: {len(vectorizer_bigram.vocabulary_)}")
print(f"Bigram training matrix shape: {X_train_bigram.shape}")

nb_bigram = MultinomialNB(alpha=1.0)
nb_bigram.fit(X_train_bigram, y_train)


Bigram vocabulary size: 136259
Bigram training matrix shape: (1200, 136259)


MultinomialNB()

#### Testing phase
As before, calculate the performance on your test data, and notice the difference with the previous

In [ ]:
### Student code here ###

# bigram, our implementation
model = NgramModalNaiveBayes(alpha=1.0)
vectorizer = CountVectorizerOff(ngram_range=(2, 2))

X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy = sum(predictions == y_test) / len(y_test)*100
print(f"OUT IMPLEMETATION Accuracy of bigram : {accuracy:.2f}%")


X_test_bigram = vectorizer_bigram.transform(test_data['Review'])

y_pred_bigram = nb_bigram.predict(X_test_bigram)

accuracy_bigram = accuracy_score(y_test, y_pred_bigram)
print(f"OFFICIAL Bigram model accuracy: {accuracy_bigram*100:}%")

print(f"\nComparison:")
print(f"Unigram accuracy: {accuracy_basic:}%")
print(f"Bigram accuracy:  {accuracy_bigram*100:}%")
print(f"Difference: {accuracy_bigram*100- accuracy_basic:+}%")


OUT IMPLEMETATION Accuracy of bigram : 89.00%
OFFICIAL Bigram model accuracy: 89.0%

Comparison:
Unigram accuracy: 85.0%
Bigram accuracy:  89.0%
Difference: +4.0%


### Trigrams
When I asked students how to improve the classification performance on this dataset, the first question was always "use trigrams" (or even higher-order n-grams). Let's try how much of an improvement that would be, by training a trigram model and testing it.

In [30]:
### Student code here ###

# our implementation
model = NgramModalNaiveBayes(alpha=1.0)
vectorizer = CountVectorizerOff(ngram_range=(3,3))

X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
model.fit(X_train_vectors, y_train)
predictions = model.predict(X_test_vectors)
accuracy = sum(predictions == y_test) / len(y_test)*100
print(f"OUR IMPLEMENTATION Accuracy of trigram : {accuracy:.2f}%")


vectorizer_trigram = CountVectorizerOff(ngram_range=(3, 3))  # Only trigrams

X_train_trigram = vectorizer_trigram.fit_transform(data['Review'])
X_test_trigram = vectorizer_trigram.transform(test_data['Review'])

print(f"Trigram vocabulary size: {len(vectorizer_trigram.vocabulary_)}")
print(f"Bigram vocabulary size: {len(vectorizer_bigram.vocabulary_)}")

nb_trigram = MultinomialNB(alpha=1.0)
nb_trigram.fit(X_train_trigram, y_train)
y_pred_trigram = nb_trigram.predict(X_test_trigram)
accuracy_trigram = accuracy_score(y_test, y_pred_trigram)

print(f"OFFICIAL Trigram model accuracy: {accuracy_trigram*100:}%")

print(f"\nResults:")
print(f"Unigram accuracy:  {accuracy_basic:}%")
print(f"Bigram accuracy:   {accuracy_bigram*100:}%")
print(f"Trigram accuracy:  {accuracy_trigram*100:}%")

OUR IMPLEMENTATION Accuracy of trigram : 77.00%
Trigram vocabulary size: 230924
Bigram vocabulary size: 136259
OFFICIAL Trigram model accuracy: 77.0%

Results:
Unigram accuracy:  85.0%
Bigram accuracy:   89.0%
Trigram accuracy:  77.0%


COMMENT : Bigram outperforms unigram by 4pp, and trigram performs the worst. Trigram has a larger vocabulary size that the bigram and unigram, leading to a sparser training matrix. We think sparsity causes lower accuracy.